In [1]:
# setting device on GPU if available, else CPU
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')


Using device: cuda

NVIDIA GeForce RTX 2080 Ti
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [2]:
!nvcc --version

/bin/bash: nvcc: command not found


In [3]:
pwd

'/coc/scratch/sanisetty3/music_motion/TGM3D'

In [4]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [5]:
import wandb

import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import DataLoader

from torch.utils import data


import copy
import os
import random
import cv2
import numpy as np
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import functools
from tqdm import tqdm
from datetime import datetime
import numpy as np
# from core.datasets.vqa_motion_dataset import VQMotionDataset,DATALoader,VQVarLenMotionDataset,MotionCollator
from einops import rearrange, reduce, pack, unpack
import sys

In [6]:
from typing import Optional, List, Dict, Tuple

In [7]:
from configs.config import cfg, get_cfg_defaults
from core.datasets.dataset_loading_utils import load_dataset
from core.datasets.vq_dataset import DATALoader
from utils.vis_utils import plot_3d_global


In [8]:
cfg = get_cfg_defaults()

In [9]:
cfg.merge_from_file("/srv/hays-lab/scratch/sanisetty3/music_motion/TGM3D/checkpoints/convq_256_1024_affine/convq_256_1024_affine.yaml")

## Dataloader

In [10]:
render_ds, sampler_val, weights_val = load_dataset(
                ["t2m", "aist", "cm"],cfg, "render"
            )

100%|██████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 1085.96it/s]


Total number of motions 6


100%|███████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 505.89it/s]


Total number of motions 8


100%|███████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 268.75it/s]

Total number of motions 7


In [11]:
render_dl = DATALoader(
    
            render_ds, batch_size=1, shuffle=False
        )

In [12]:
batch = (next(iter(render_dl)))
    
print(batch["names"])

['00_000048']


In [13]:
batch["motion"].shape

torch.Size([1, 80, 271])

In [14]:
from utils.vis_utils.render_final import Renderer

In [15]:
renderer = Renderer()

In [16]:
renderer.render(render_ds.datasets[0]
                    .inv_transform(batch["motion"])
                    .squeeze()
                    .float(), "test" , 0 , "/srv/hays-lab/scratch/sanisetty3/music_motion/TGM3D/renders")

## HML save txt

In [168]:
for i in ind.flatten().tolist():
    new_dict[f"{i}"] +=1

In [169]:
[v for k,v in ]

{'0': 0,
 '1': 0,
 '2': 0,
 '3': 0,
 '4': 2,
 '5': 0,
 '6': 0,
 '7': 0,
 '8': 0,
 '9': 0,
 '10': 0,
 '11': 0,
 '12': 0,
 '13': 0,
 '14': 0,
 '15': 0,
 '16': 0,
 '17': 0,
 '18': 0,
 '19': 0,
 '20': 1,
 '21': 0,
 '22': 0,
 '23': 1,
 '24': 0,
 '25': 0,
 '26': 0,
 '27': 0,
 '28': 0,
 '29': 0,
 '30': 0,
 '31': 0,
 '32': 0,
 '33': 0,
 '34': 0,
 '35': 0,
 '36': 0,
 '37': 0,
 '38': 0,
 '39': 0,
 '40': 0,
 '41': 0,
 '42': 0,
 '43': 2,
 '44': 0,
 '45': 0,
 '46': 0,
 '47': 0,
 '48': 0,
 '49': 0,
 '50': 0,
 '51': 0,
 '52': 2,
 '53': 0,
 '54': 0,
 '55': 0,
 '56': 0,
 '57': 0,
 '58': 0,
 '59': 0,
 '60': 0,
 '61': 0,
 '62': 0,
 '63': 0,
 '64': 0,
 '65': 0,
 '66': 0,
 '67': 0,
 '68': 1,
 '69': 0,
 '70': 0,
 '71': 0,
 '72': 0,
 '73': 0,
 '74': 0,
 '75': 0,
 '76': 0,
 '77': 0,
 '78': 0,
 '79': 0,
 '80': 0,
 '81': 0,
 '82': 0,
 '83': 0,
 '84': 0,
 '85': 0,
 '86': 0,
 '87': 0,
 '88': 0,
 '89': 0,
 '90': 0,
 '91': 0,
 '92': 0,
 '93': 0,
 '94': 0,
 '95': 0,
 '96': 0,
 '97': 1,
 '98': 0,
 '99': 0,
 '100': 0,

In [141]:
22*9-22*12

-66

In [98]:
hml_paths = sorted(glob("/srv/hays-lab/scratch/sanisetty3/music_motion/HumanML3D/HumanML3D_SMPL/new_joints_vec/*.npy"))


In [99]:
og_hml_train = []
f = open("/srv/hays-lab/scratch/sanisetty3/music_motion/HumanML3D/HumanML3D/train.txt", "r")
for l in f:
    og_hml_train.append(l.rstrip()[-6:])
    
og_hml_train = np.unique(og_hml_train)
    
og_hml_test = []
f = open("/srv/hays-lab/scratch/sanisetty3/music_motion/HumanML3D/HumanML3D/test.txt", "r")
for l in f:
    og_hml_test.append(l.rstrip()[-6:])
    
og_hml_test = np.unique(og_hml_test)

    
og_hml_val = []
f = open("/srv/hays-lab/scratch/sanisetty3/music_motion/HumanML3D/HumanML3D/val.txt", "r")
for l in f:
    og_hml_val.append(l.rstrip()[-6:])

og_hml_val = np.unique(og_hml_val)


In [100]:
len(og_hml_train) + len(og_hml_test) + len(og_hml_val)

14614

In [101]:
new_hml = sorted(np.unique([os.path.basename(path).split("_")[-1].split(".")[0] for path in hml_paths]))
len(new_hml)

15227

In [102]:
train_names = sorted(list(set(new_hml).intersection(og_hml_train)))
test_names = sorted(list(set(new_hml).intersection(og_hml_test)))
val_names = sorted(list(set(new_hml).intersection(og_hml_val)))
additional_hml_names = sorted(list(set(new_hml) ^ (set(list(train_names) + list(test_names) + list(val_names)))))

In [103]:
len(train_names) + len(test_names) + len(val_names)  + len(additional_hml_names)

15227

In [104]:
all_train  =  sorted([os.path.basename(path).split(".")[0] for path in hml_paths if os.path.basename(path).split("_")[-1].split(".")[0] in train_names])
all_test  =  sorted([os.path.basename(path).split(".")[0] for path in hml_paths if os.path.basename(path).split("_")[-1].split(".")[0] in test_names])
all_val  =  sorted([os.path.basename(path).split(".")[0] for path in hml_paths if os.path.basename(path).split("_")[-1].split(".")[0] in val_names])
additional_hml_names  =  sorted([os.path.basename(path).split(".")[0] for path in hml_paths if os.path.basename(path).split("_")[-1].split(".")[0] in additional_hml_names])

In [105]:
len(all_train) + len(all_test) + len(all_val) + len(additional_hml_names)

45629

In [106]:
all_train = sorted(all_train + additional_hml_names[:5000])
all_test = sorted(all_test + additional_hml_names[5000:])

In [92]:
# with open("/srv/hays-lab/scratch/sanisetty3/music_motion/HumanML3D/HumanML3D_SMPL/train.txt" , 'w') as f:
#     for line in all_train:
#         f.write(line + "\n") 

45629

## Save CM

In [18]:
cm_paths  = sorted(glob("/srv/hays-lab/scratch/sanisetty3/music_motion/Choreomaster_SMPL/new_joints_vec/*npy"))
len(cm_paths)

108

In [7]:
cm_names = sorted(np.unique([os.path.basename(path).split("_")[-1].split(".")[0] for path in cm_paths]))
len(cm_names)

36

In [11]:
train_names , test_names, val_names, _  = np.split(np.random.permutation((cm_names)) , [29 ,33 ,36])

In [17]:
len(train_names)

29

In [14]:
all_train  =  sorted([os.path.basename(path).split(".")[0] for path in cm_paths if os.path.basename(path).split("_")[-1].split(".")[0] in train_names])
all_test  =  sorted([os.path.basename(path).split(".")[0] for path in cm_paths if os.path.basename(path).split("_")[-1].split(".")[0] in test_names])
all_val  =  sorted([os.path.basename(path).split(".")[0] for path in cm_paths if os.path.basename(path).split("_")[-1].split(".")[0] in val_names])
all_ = sorted(all_train + all_test + all_val )

In [16]:
len(all_)

108

In [22]:
with open("/srv/hays-lab/scratch/sanisetty3/music_motion/Choreomaster_SMPL/test.txt" , 'w') as f:
    for line in all_test:
        f.write(line + "\n") 

## Save AIST

In [23]:
aist_paths = sorted(glob("/srv/hays-lab/scratch/sanisetty3/music_motion/AIST_SMPL/new_joints_vec/*.npy"))


In [24]:
# [os.remove(p) for p in aist_paths if "_M_" in p]

In [25]:
len(aist_paths)

4087

In [38]:
anno_dir = "/srv/share/datasets/AIST/aist_plusplus_final/"

seq_names = []
seq_names += np.loadtxt(
    os.path.join("/srv/share/datasets/AIST/aist_plusplus_final/splits/all.txt"), dtype=str
).tolist()
ignore_list = np.loadtxt(
os.path.join(anno_dir, "ignore_list.txt"), dtype=str
).tolist()
seq_names = [name for name in seq_names if name not in ignore_list]


In [39]:
len(seq_names)

1363

In [40]:
all_perm  =  sorted([os.path.basename(path).split(".")[0] for path in aist_paths if os.path.basename(path).split(".")[0][-26:] in seq_names])
len(all_perm)

4087

In [41]:
with open("/srv/hays-lab/scratch/sanisetty3/music_motion/AIST_SMPL/all.txt" , 'w') as f:
    for line in all_perm:
        f.write(line + "\n") 